In [ ]:
from pystac_client import Client
from odc.stac import load
from dea_tools.coastal import pixel_tides

from dask.distributed import Client as DaskClient

import numpy as np
from pathlib import Path
import odc.geo.xr  # noqa

In [ ]:
# STAC Catalog URL
catalog = "https://earth-search.aws.element84.com/v1"

# Create a STAC Client
client = Client.open(catalog)

# Set up Dask
dask_client = DaskClient()
dask_client

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
coords = 20.774852340215258, 106.77857399045257  # Near Haiphong
buffer = 0.1
bbox = (coords[1] - buffer, coords[0] - buffer, coords[1] + buffer, coords[0] + buffer)

datetime = "2023/2024"

# Tide data and config
home = Path("~")
tide_data_location = f"{home}/tide_models"

In [ ]:
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime=datetime
).item_collection()

print(f"Found {len(items)} STAC items")

In [ ]:
data = load(
    items,
    bands=["red", "green", "blue", "cloud", "scl"],
    bbox=bbox,
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)

data

In [ ]:
# cloud_mask = data.cloud > 25  # Percentage likelihood a pixel is cloud
NODATA = 0
CLOUD_SHADOW = 3
CLOUD_MEDIUM_PROBABILITY = 8
CLOUD_HIGH_PROBABILITY = 9

mask = data.scl.isin([NODATA, CLOUD_SHADOW, CLOUD_MEDIUM_PROBABILITY, CLOUD_HIGH_PROBABILITY])

masked = data.where(~mask, other=np.nan)
masked = masked.drop_vars("scl")

In [ ]:
# We're doing a lowres tide model, as we are just going to filter out scenes
tides_lowres = pixel_tides(
    masked, resample=False, directory=tide_data_location, model="FES2022", dask_compute=True
)

In [ ]:
lowest, highest = tides_lowres.quantile([0.3, 0.7]).values

low_scenes = tides_lowres.where(tides_lowres < lowest, drop=True)
high_scenes = tides_lowres.where(tides_lowres > highest, drop=True)

data_low = masked.sel(time=low_scenes.time)
data_high = masked.sel(time=high_scenes.time)

print(f"Found {len(data_low.time)} low tide days and {len(data_high.time)} high tide days out of {len(data.time)} days")

In [ ]:
median_low = data_low.median("time").compute()
median_high = data_high.median("time").compute()

In [ ]:
median = masked.median("time").compute()
median.odc.explore(vmin=1000, vmax=4000)

In [ ]:
median_high.odc.explore(vmin=1000, vmax=4000)

In [ ]:
median_low.odc.explore(vmin=1000, vmax=4000)

In [ ]:
# TODO: consider adding MNDWI here, as an intro into the next day's work doing coastline extraction